In [1]:
import os
import numpy as np
from tensorflow.keras.applications import EfficientNetB7

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
data_dir = '/Users/saikarthik/Downloads/untitled folder/parkinsons_dataset'


In [3]:
img_height, img_width = 224, 224 
batch_size = 32


In [4]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,  
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)


In [5]:

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
)

Found 665 images belonging to 2 classes.


In [6]:

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
)

Found 166 images belonging to 2 classes.


In [7]:
base_model = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

2025-04-02 12:18:13.049654: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-04-02 12:18:13.049684: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-02 12:18:13.049691: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-02 12:18:13.049722: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-02 12:18:13.049735: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


258076736/258076736 [==============================] - 77s 0us/step


In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [9]:
for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


In [10]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
)

Epoch 1/100


2025-04-02 12:19:42.796316: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - ETA: 0s - loss: 0.6406 - accuracy: 0.6932

2025-04-02 12:20:04.723207: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - 39s 1s/step - loss: 0.6406 - accuracy: 0.6932 - val_loss: 0.5793 - val_accuracy: 0.7349
Epoch 2/100
21/21 [==============================] - 14s 602ms/step - loss: 0.5996 - accuracy: 0.7353 - val_loss: 0.5790 - val_accuracy: 0.7349
Epoch 3/100
21/21 [==============================] - 13s 613ms/step - loss: 0.6057 - accuracy: 0.7263 - val_loss: 0.5790 - val_accuracy: 0.7349
Epoch 4/100
21/21 [==============================] - 13s 602ms/step - loss: 0.5881 - accuracy: 0.7338 - val_loss: 0.5789 - val_accuracy: 0.7349
Epoch 5/100
21/21 [==============================] - 13s 609ms/step - loss: 0.6101 - accuracy: 0.7308 - val_loss: 0.5826 - val_accuracy: 0.7349
Epoch 6/100
21/21 [==============================] - 13s 607ms/step - loss: 0.5953 - accuracy: 0.7293 - val_loss: 0.5780 - val_accuracy: 0.7349
Epoch 7/100
21/21 [==============================] - 13s 612ms/step - loss: 0.5991 - accuracy: 0.7338 - val_loss: 0.5838 - val_accuracy: 0.7349
Epoch 8

In [ ]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

validation_generator.reset()
predictions = (model.predict(validation_generator) > 0.5).astype("int64")
true_classes = validation_generator.classes

print("\nClassification Report:")
print(classification_report(true_classes, predictions))

print("\nConfusion Matrix:")
print(confusion_matrix(true_classes, predictions))

NameError: name 'model' is not defined